In [2]:
from sys import path
path.append("/home/ec2-user/SageMaker/data-science-development/utils")
path.append("/home/ec2-user/SageMaker/data-science-development/config")

import pandas as pd
import numpy as np

from collections import defaultdict
from tqdm import tqdm 
from heapq import nlargest

tqdm.pandas()

In [3]:
df_pred = pd.read_csv("../Data/df_pred_ext.csv").drop("Unnamed: 0", axis=1)

In [4]:
career_paths = df_pred.groupby("candidate_id")["isco_code4"]

In [5]:
# Convert to 2d-arrays, getting rid of candidate_ids as values
career_paths = career_paths.progress_apply(lambda x: x.values)

100%|██████████| 469568/469568 [00:11<00:00, 41203.50it/s]


In [6]:
career_lens = career_paths.apply(len)
career_paths = career_paths.loc[(career_lens > 1)]

In [7]:
career_paths = career_paths.loc[career_paths.apply(lambda x: x[-1] != x[-2])]
len(career_paths)

113724

In [13]:
career_paths_train = career_paths.iloc[:int(0.8 * 113724)]
career_paths_test = career_paths.iloc[int(0.8 * 113724):]

In [14]:
switches = defaultdict(lambda: defaultdict(int))

for c, jobs in career_paths_train.iteritems():
    for i in range(len(jobs)):
        if i < len(jobs) - 1 and jobs[i] != jobs[i + 1]:
            switches[jobs[i]][jobs[i  + 1]] += 1

In [51]:
most_common = pd.Series({k: set(nlargest(5, v, key=v.get)) for k, v in switches.items()})

In [52]:
most_common.head()

208    {229, 344, 345, 348, 189}
348    {229, 207, 208, 344, 345}
344    {323, 195, 117, 343, 345}
345    {323, 195, 332, 177, 344}
185    {195, 227, 177, 344, 345}
dtype: object

In [53]:
most_common_switch = career_paths_test.progress_apply(lambda x: x[-1] in most_common.loc[x[-2]]).mean()

100%|██████████| 22745/22745 [00:00<00:00, 38877.47it/s]


In [54]:
most_common_switch, 1.96 * np.sqrt(((1 - most_common_switch)* (most_common_switch)) / (113724*0.2))

(0.46564080017586285, 0.006482718581574373)

In [55]:
print(f"Most common switch accuracy: {most_common_switch * 100:>.2f}%")

Most common switch accuracy: 46.56%
